Import necessary libraries

In [1]:
from langchain_ollama import OllamaLLM,OllamaEmbeddings

d:\python\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialize the LLM

In [2]:
# LLM_MODEL = 'llama3.2:1b'
LLM_MODEL = 'qwen3:1.7b'

llm = OllamaLLM(model=LLM_MODEL)

In [3]:
import os
import json
from langchain_core.documents import Document

DATA_DIR = os.path.join(os.getcwd(), "Data")

def load_json_docs():
    docs = []

    for file in os.listdir(DATA_DIR):
        if file.endswith(".json"):              
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

                products = data.get("products", [])
                orders = data.get("orders", [])
                faqs = data.get("faqs",[])

                # Products
                for product in products:
                    text = json.dumps(product, indent=2)

                    docs.append(
                        Document(
                            page_content=text,
                            metadata={
                                "type": "product",
                                "id": product["id"],
                                "name": product["name"],
                                "category": product["category"],
                            }
                        )
                    )

                # FAQs
                for faq in faqs:
                    text = json.dumps(faq, indent=2)
                    docs.append(
                        Document(
                            page_content=text,
                            metadata={
                                "type": "faq",
                                "faq_id": faq.get("id"),
                                "question": faq.get("question"),
                            }
                        )
                    )

                # Orders
                for order in orders:
                    text = json.dumps(order, indent=2)
                    docs.append(
                        Document(
                            page_content=text,
                            metadata={
                                "type": "order",
                                "order_id": order.get("order_id"),
                                "customer_name": order.get("customer", {}).get("customer_name"),
                                "status": order.get("status")
                            }
                        )
                    )

    return docs

raw_docs = load_json_docs()
print(len(raw_docs))
print(raw_docs)

84
[Document(metadata={'type': 'faq', 'faq_id': 'faq_001', 'question': 'How can I check the status of my order?'}, page_content='{\n  "id": "faq_001",\n  "question": "How can I check the status of my order?",\n  "answer": "You can check the status by entering your order ID in the order tracking section or asking the chatbot."\n}'), Document(metadata={'type': 'faq', 'faq_id': 'faq_002', 'question': 'What payment methods do you accept?'}, page_content='{\n  "id": "faq_002",\n  "question": "What payment methods do you accept?",\n  "answer": "We accept credit cards, debit cards, bank transfers, and mobile payments."\n}'), Document(metadata={'type': 'faq', 'faq_id': 'faq_003', 'question': 'Do you offer cash on delivery?'}, page_content='{\n  "id": "faq_003",\n  "question": "Do you offer cash on delivery?",\n  "answer": "Yes, cash on delivery is available for selected locations."\n}'), Document(metadata={'type': 'faq', 'faq_id': 'faq_004', 'question': 'What is your return policy?'}, page_con

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50
)

documents = splitter.split_documents(raw_docs)
print(len(documents))
print(documents)

123
[Document(metadata={'type': 'faq', 'faq_id': 'faq_001', 'question': 'How can I check the status of my order?'}, page_content='{\n  "id": "faq_001",\n  "question": "How can I check the status of my order?",\n  "answer": "You can check the status by entering your order ID in the order tracking section or asking the chatbot."\n}'), Document(metadata={'type': 'faq', 'faq_id': 'faq_002', 'question': 'What payment methods do you accept?'}, page_content='{\n  "id": "faq_002",\n  "question": "What payment methods do you accept?",\n  "answer": "We accept credit cards, debit cards, bank transfers, and mobile payments."\n}'), Document(metadata={'type': 'faq', 'faq_id': 'faq_003', 'question': 'Do you offer cash on delivery?'}, page_content='{\n  "id": "faq_003",\n  "question": "Do you offer cash on delivery?",\n  "answer": "Yes, cash on delivery is available for selected locations."\n}'), Document(metadata={'type': 'faq', 'faq_id': 'faq_004', 'question': 'What is your return policy?'}, page_co

In [5]:
from langchain_community.vectorstores import Chroma

EMBEDDING_MODEL = 'nomic-embed-text'

CHORMA_DB_DIR = os.path.join(os.getcwd(), "Vectorstore")

embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
vectordb = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory=CHORMA_DB_DIR
)

vectordb.persist()

print("Vector DB Created.")

Vector DB Created.


C:\Users\Dulitha Harasara\AppData\Local\Temp\ipykernel_8916\3698220226.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
You are an intelligent customer support chatbot for an online store.
You must answer every question ONLY using the information provided in the context.

────────────────────────────────────────
GENERAL RESPONSE RULES
────────────────────────────────────────
1. Provide answers strictly from the context. Never use external knowledge.
2. For product recommendations, FAQs, or order-related questions, respond ONLY using context data.
3. If the information is missing, reply:
   "Sorry, I don't have information about that."
4. Do NOT invent, assume, or hallucinate any product, price, specification, or store detail.
5. Always respond politely, clearly, and professionally.
6. If the user asks about unrelated topics (jokes, math, coding, etc.), reply:
   "I am an online store assistant. I can help you with product information, orders, and FAQs."
7. If the question is unclear or incomplete, ask for clarification.

────────────────────────────────────────
PRODUCT & RECOMMENDATION RULES
────────────────────────────────────────
8. Recommend products WITH full details, but only if they exist in the context.
9. For comparisons or alternatives, use ONLY the product details available in the context.
10. For price, availability, specifications, or features, rely ONLY on context information.

────────────────────────────────────────
GREETING HANDLING RULE
────────────────────────────────────────
11. If the user sends ONLY a greeting (e.g., "hi", "hello", "hey"), respond:
    "<same greeting> How can I assist you today?"
    Do not treat unrelated words that resemble greetings as greetings.

────────────────────────────────────────
PRODUCT CATEGORY / STORE CONTENT RULE
────────────────────────────────────────
12. If the user asks what products or categories the store has,
    list ONLY the product names or categories found in the context.

────────────────────────────────────────
NEUTRAL RESPONSE RULE
────────────────────────────────────────
13. For neutral acknowledgments ("ok", "hmm", "fine", "sure", etc.), reply:
    "Sure! Let me know what you would like to do next."

────────────────────────────────────────
FAQ HANDLING RULE
────────────────────────────────────────
14. If the user's question matches an FAQ entry, respond with the EXACT answer from the context.
15. If no matching FAQ exists, reply:
    "Sorry, I don't have information about that."

────────────────────────────────────────
ORDER HANDLING RULE
────────────────────────────────────────
16. For order-related questions (status, tracking, order details), use ONLY the context.
17. Identify orders using:
    – order_id provided by the user, or
    – customer name provided by the user.
18. When an order is found, show ONLY the details available in the context:
    – order_id
    – products
    – status
    – order_date
    – expected_delivery
19. If no matching order exists, reply:
    "Sorry, I can't find any order with those details."
20. Never assume missing order details.

────────────────────────────────────────
END OF RULES
────────────────────────────────────────

User Question:
{question}

Context:
{context}
"""


prompt = ChatPromptTemplate.from_messages([("human",message)])
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are an intelligent customer support chatbot for an online store.\nYou must answer every question ONLY using the information provided in the context.\n\n────────────────────────────────────────\nGENERAL RESPONSE RULES\n────────────────────────────────────────\n1. Provide answers strictly from the context. Never use external knowledge.\n2. For product recommendations, FAQs, or order-related questions, respond ONLY using context data.\n3. If the information is missing, reply:\n   "Sorry, I don\'t have information about that."\n4. Do NOT invent, assume, or hallucinate any product, price, specification, or store detail.\n5. Always respond politely, clearly, and professionally.\n6. If the user asks about unrelated topics (jokes, math, coding, e

In [7]:
CHORMA_DB_DIR = os.path.join(os.getcwd(), "Vectorstore")

db = Chroma(
    persist_directory=CHORMA_DB_DIR,
    embedding_function=embeddings
)

retriever = db.as_retriever(search_kwargs={"k": 4})

C:\Users\Dulitha Harasara\AppData\Local\Temp\ipykernel_8916\1621540715.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(


In [8]:
chain = ({"context":retriever,"question":RunnablePassthrough()} | prompt | llm)

In [9]:
response = chain.invoke("I want to know about AsterPhone Pro 14 Plus?")
print(response)

The AsterPhone Pro 14 Plus is a flagship smartphone from the Aster brand. It features a 6.7-inch LTPO OLED display, a Aster A18 processor, 12GB RAM, and storage options of 256GB and 512GB. The camera includes a 50MP main lens, a 12MP ultra-wide lens, and a 12MP depth lens. It has a 4800mAh battery. The phone is priced at $1199 USD, available in stock with 520 units. It was released on September 15, 2024, and comes with a 24-month warranty. The product is tagged as "flagship," "5G," "OLED," and "fast-charge."


In [10]:
# response = chain.invoke("what is a car?")
# print(response) 



import gradio as gr

chain = ({"context":retriever,"question":RunnablePassthrough()} | prompt | llm)

def yes(message, history):
    response = chain.invoke(message)
    return response

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value["value"])
    else:
        print("You downvoted this response: " + data.value["value"])

with gr.Blocks() as demo:
    # chatbot = gr.Chatbot(placeholder="<strong>Your Personal Yes-Man</strong><br>Ask Me Anything")
    # chatbot.like(vote, None, None)
    gr.ChatInterface(fn=yes),
    height=1600,
    
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
